In [1]:
%load_ext Cython
%load_ext autoreload
%autoreload 2

import dask
dask.config.set(num_workers=8)

import satpy
satpy.config.set({'cache_dir': "D:/sat_data/cache/"})
satpy.config.set({'cache_sensor_angles': True})
satpy.config.set({'cache_lonlats': True})

from datetime import datetime
from satpy import Scene
import dask.array as da
from glob import glob

from pyfires.PYF_detection import stage1_tests
from pyfires.PYF_basic import *

from pyspectral.rsr_reader import RelativeSpectralResponse
import numpy as np

import warnings
warnings.filterwarnings('ignore')


In [39]:
indir = 'D:/sat_data/HIM/'

ifiles_l15 = glob(indir + '*.DAT')

ifile_cld = f'{indir}/AHI-CMSK_v1r1_h09_s202308240540213_e202308240549407_c202308240557548.nc'

odir = 'D:/sat_data/SEV_FIRE/'


rad_dict = {'vi1': 1647.567,
            'vi2': 244.192,
            'swi': 10.259}

bdict = {'vi1_band': 'B03',
         'vi2_band': 'B05',
         'swi_band': 'B07',
         'lwi_band': 'B13',
         'lw2_band': 'B11'}

blist = []
for item in bdict:
    blist.append(bdict[item])

scn = initial_load(ifiles_l15, 'ahi_hsd', bdict, rad_dict, sw_bt_thresh=270, lw_bt_thresh=260)
#scn.save_datasets(base_dir=odir, enhance=False, dtype=np.float32)

vi_rad_arr = np.array(scn['VI1_RAD'])
ndfi_arr = np.array(scn['sw_ndfi'])
vi_diff_arr = np.array(scn['VI1_DIFF'])
btd_arr = np.array(scn['BTD']) 
mir_bt_arr = np.array(scn['SWI__BT'])
#irr_arr = np.array(scn['SWI_RAD'])
lwi_bt_arr = np.array(scn['LW1__BT'])
lwi_rad_arr = np.array(scn['LW1_RAD'])
lsm = np.array(scn['LSM'])
btd_lw_arr = np.array(scn['BTD_LW']) 

cannot convert float NaN to integer


In [20]:
btd_sw_thr = 6
btd_lw_thr = 1.

pfp = da.where(btd_arr > btd_sw_thr, 1, 0)
pfp = da.where(btd_lw_arr > btd_lw_thr, pfp, 0)
save_output(scn, pfp, 'pfp', odir, 'VI1_DIFF')

In [22]:
stg1_output = stage1_tests(scn['SWI__BT'].data,
                           scn['BTD'].data,
                           scn['VI1_DIFF'].data,
                           scn['SZA'].data,
                           scn['LSM'].data,
                           ksizes=[5, 7, 9])

scn['PFP'] = scn['SWI__BT'].copy()
scn['PFP'].attrs['name'] = 'PFP'
scn['PFP'].data = stg1_output
scn.save_dataset('PFP', base_dir=odir, enhance=False, dtype=np.float32)

In [43]:
%%cython

from libc.math cimport sqrt, isnan
cimport numpy as np
import numpy as np
import cython

@cython.boundscheck(False)
@cython.cdivision(True)
@cython.wraparound(False)
@cython.initializedcheck(False)
cdef get_curwindow_pos(int xpos, int ypos, int winsize, int scn_width, int scn_height):
    """Find the correct position of the moving window in the input arrays.
    Inputs:
     - xpos: The x position of the candidate pixel
     - ypos: The y position of the candidate pixel
     - winsize: The radius of the window
     - scn_width: The width of the whole satellite scene
     - scn_height: The height of the whole satellite scene 
    Returns:
     - shape_sel: An array specifying the correct indices for the windowed region (x_0, x_1, y_0, y_1)
    """
    cdef int shape_sel[4]

    # Set up the output pixel selection
    shape_sel = [xpos - winsize, xpos + winsize + 1, ypos - winsize, ypos + winsize + 1]

    # Apply some range tests, so we can use cython in the non-bounds checking mode
    if shape_sel[0] < 0:
        shape_sel[0] = -999
    elif shape_sel[0] >= scn_width:
        shape_sel[0] = -999
    if shape_sel[1] < 0:
        shape_sel[0] = -999
    elif shape_sel[1] >= scn_height:
        shape_sel[0] = -999

    return shape_sel

@cython.boundscheck(False)
@cython.cdivision(True)
@cython.wraparound(False)
@cython.initializedcheck(False)
cdef get_window_mea_stdv(int winsize,
                         float[:,:] cur__vis,
                         float[:,:] cur_ndfi,
                         float[:,:] cur__lwi,
                         float[:,:] cur__btd,
                         float[:,:] cur__mir,
                         float[:,:] cur__vid,
                         unsigned char[:,:] cur__lsm,
                         float mir_min_thresh = 270,
                         float mir_max_thresh = 340,
                         float btd_max_thresh = 30,
                         unsigned char lsm_land_val = 2, 
                         ):
    cdef int arrsize = winsize + winsize + 1
    cdef int npix = arrsize * arrsize
    cdef int i = 0
    cdef int j = 0

    cdef float n_good = 0
    cdef float perc_good = 0
    
    cdef float sum_val_vi = 0
    cdef float mea_val_vi = -999.
    cdef float std_val_vi = -999.
    
    cdef float sum_val_nd = 0
    cdef float mea_val_nd = -999.
    cdef float std_val_nd = -999.
    
    cdef float sum_val_lw = 0
    cdef float mea_val_lw = -999.
    cdef float std_val_lw = -999.
    
    cdef float sum_val_btd = 0
    cdef float mea_val_btd = -999.
    cdef float std_val_btd = -999.
    
    cdef float sum_val_mir = 0
    cdef float mea_val_mir = -999.
    cdef float std_val_mir = -999.
    
    cdef float sum_val_vid = 0
    cdef float mea_val_vid = -999.
    cdef float std_val_vid = -999.
    
    cdef float res[13]

 
    # Loop to find mean
    for i in range(0, arrsize):
        for j in range(0, arrsize):
            if i >= winsize - 1 and i <= winsize + 1 and j >= winsize - 1 and j <= winsize + 1:
                continue
            
            if cur__mir[i,j] < mir_min_thresh or cur__mir[i,j] > mir_max_thresh:
                continue
            if cur__btd[i,j] > btd_max_thresh:
                continue
            if cur__lsm[i,j] != lsm_land_val:
                continue
            sum_val_vi = sum_val_vi + cur__vis[i, j]
            sum_val_nd = sum_val_nd + cur_ndfi[i, j]
            sum_val_lw = sum_val_lw + cur__lwi[i, j]
            sum_val_btd = sum_val_btd + cur__btd[i, j]
            sum_val_mir = sum_val_mir + cur__mir[i, j]
            sum_val_vid = sum_val_vid + cur__vid[i, j]
            n_good = n_good + 1
    perc_good = n_good / (npix - 9)
    
    mea_val_vi = sum_val_vi / n_good
    mea_val_nd = sum_val_nd / n_good
    mea_val_lw = sum_val_lw / n_good
    mea_val_btd = sum_val_btd / n_good
    mea_val_mir = sum_val_mir / n_good
    mea_val_vid = sum_val_vid / n_good
    
     # Loop to find standard deviation
    sum_val_vi = 0
    sum_val_nd = 0
    sum_val_lw = 0
    sum_val_btd = 0
    sum_val_mir = 0
    sum_val_vid = 0
    
    for i in range(0, arrsize):
        for j in range(0, arrsize):
            if i >= winsize - 1 and i <= winsize + 1 and j >= winsize - 1 and j <= winsize + 1:
                continue
            if cur__mir[i,j] < mir_min_thresh or cur__mir[i,j] > mir_max_thresh:
                continue
            if cur__btd[i,j] > btd_max_thresh:
                continue
            if cur__lsm[i,j] != lsm_land_val:
                continue
                
            sum_val_vi = sum_val_vi + (cur__vis[i, j] - mea_val_vi) * (cur__vis[i, j] - mea_val_vi)
            sum_val_nd = sum_val_nd + (cur_ndfi[i, j] - mea_val_nd) * (cur_ndfi[i, j] - mea_val_nd)
            sum_val_lw = sum_val_lw + (cur__lwi[i, j] - mea_val_lw) * (cur__lwi[i, j] - mea_val_lw)
            sum_val_btd = sum_val_btd + (cur__btd[i, j] - mea_val_btd) * (cur__btd[i, j] - mea_val_btd)
            sum_val_mir = sum_val_mir + (cur__mir[i, j] - mea_val_mir) * (cur__mir[i, j] - mea_val_mir)
            sum_val_vid = sum_val_vid + (cur__vid[i, j] - mea_val_vid) * (cur__vid[i, j] - mea_val_vid)
            
    std_val_lw = sqrt(sum_val_lw / n_good)
    std_val_nd = sqrt(sum_val_nd / n_good)
    std_val_vi = sqrt(sum_val_vi / n_good)
    std_val_btd = sqrt(sum_val_btd / n_good)
    std_val_mir = sqrt(sum_val_mir / n_good)
    std_val_vid = sqrt(sum_val_vid / n_good)
    
    res[0] = perc_good
    res[1] = mea_val_lw
    res[2] = std_val_lw
    res[3] = mea_val_nd
    res[4] = std_val_nd
    res[5] = mea_val_vi
    res[6] = std_val_vi
    res[7] = mea_val_btd
    res[8] = std_val_btd
    res[9] = mea_val_mir
    res[10] = std_val_mir
    res[11] = mea_val_vid
    res[12] = std_val_vid
    return res 


def get_mea_std_window(float[:,:] vi_rad_data,
                       float[:,:] ndfi_data,
                       float[:,:] lw_rad_data,
                       float[:,:] btd_arr,
                       float[:,:] mir_arr,
                       float[:,:] vid_arr,
                       unsigned char[:,:] lsm,                       
                       int winsize):
    
    cdef int scn_width = vi_rad_data.shape[0]
    cdef int scn_height = vi_rad_data.shape[1]
    
    cdef int x_0 = 0
    cdef int x_1 = scn_width
    cdef int y_0 = 0
    cdef int y_1 = scn_height
    
    # Counters for main loop
    cdef int x = 0
    cdef int y = 0
    cdef int wsize = winsize
    cdef int cur_win[4]
    
    # Intermediate variables
    cdef float sum_tot = 0
    cdef int n = 0
    cdef float cen_vid = 0
    cdef float cen_btd = 0
    cdef float cen_mir = 0
    cdef float cen_irr = 0
    cdef float cen_lwi = 0
    cdef float res[13] 

    # Output datasets
    cdef np.ndarray[dtype=np.float32_t, ndim=3] outarr = np.zeros((scn_width, scn_height, 13), dtype=np.single)
    cdef float[:,:, ::1] outarr_view = outarr
    
    wsize = 15
    
    # Loop across all pixels in the image
    for x in range(x_0, x_1):
        for y in range(y_0, y_1):
            # Determine coordinates of the current window
            cur_win = get_curwindow_pos(x, y, wsize, scn_width, scn_height)
            # Don't process if any coords are bad (usually at extreme edges of image)
            if cur_win[0] < 0:
                continue

            cur__vis = vi_rad_data[cur_win[0]:cur_win[1], cur_win[2]:cur_win[3]]
            cur_ndfi = ndfi_data[cur_win[0]:cur_win[1], cur_win[2]:cur_win[3]]
            cur__lwi = lw_rad_data[cur_win[0]:cur_win[1], cur_win[2]:cur_win[3]]
            cur__btd = btd_arr[cur_win[0]:cur_win[1], cur_win[2]:cur_win[3]]
            cur__mir = mir_arr[cur_win[0]:cur_win[1], cur_win[2]:cur_win[3]]
            cur__vid = vid_arr[cur_win[0]:cur_win[1], cur_win[2]:cur_win[3]]
            cur__lsm = lsm[cur_win[0]:cur_win[1], cur_win[2]:cur_win[3]]

            # Compute mean and standard deviation across window
            res = get_window_mea_stdv(wsize,
                                      cur__vis,
                                      cur_ndfi,
                                      cur__lwi,
                                      cur__btd,
                                      cur__mir,
                                      cur__vid,
                                      cur__lsm)

            # Set output array to:
            # [0]: Percentage of acceptable pixels in window
            outarr_view[x, y, 0] = res[0]
            outarr_view[x, y, 1] = res[1]
            outarr_view[x, y, 2] = res[2]
            outarr_view[x, y, 3] = res[3]
            outarr_view[x, y, 4] = res[4]
            outarr_view[x, y, 5] = res[5]
            outarr_view[x, y, 6] = res[6]
            outarr_view[x, y, 7] = res[7]
            outarr_view[x, y, 8] = res[8]
            outarr_view[x, y, 9] = res[9]
            outarr_view[x, y, 10] = res[10]
            outarr_view[x, y, 11] = res[11]
            outarr_view[x, y, 12] = res[12]
        
    return outarr    

Content of stdout:
_cython_magic_8abd8d2268520929da0567858ed7552b36b1b080.c
C:\Users\Simon\mambaforge\Lib\site-packages\numpy\core\include\numpy\npy_1_7_deprecated_api.h(14) : Warning Msg: Using deprecated NumPy API, disable it with #define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION
C:\Users\Simon\.ipython\cython\_cython_magic_8abd8d2268520929da0567858ed7552b36b1b080.c(21180): warning C4244: '=': conversion from 'double' to 'float', possible loss of data
C:\Users\Simon\.ipython\cython\_cython_magic_8abd8d2268520929da0567858ed7552b36b1b080.c(21562): warning C4244: '=': conversion from 'double' to 'float', possible loss of data
C:\Users\Simon\.ipython\cython\_cython_magic_8abd8d2268520929da0567858ed7552b36b1b080.c(21571): warning C4244: '=': conversion from 'double' to 'float', possible loss of data
C:\Users\Simon\.ipython\cython\_cython_magic_8abd8d2268520929da0567858ed7552b36b1b080.c(21580): warning C4244: '=': conversion from 'double' to 'float', possible loss of data
C:\Users\Simon\.i

In [44]:
st_time = datetime.utcnow()
outa = get_mea_std_window(vi_rad_arr,    # VIS chan
                          ndfi_arr,      # NDFI
                          lwi_rad_arr,   # LW Radiance
                          btd_arr,       # MIR-LW BTD
                          mir_bt_arr,    # MIR BT
                          vi_diff_arr,   # MIR-LWIR-VIS radiance diff
                          lsm,           # The land-sea mask
                          15)
en_time = datetime.utcnow()

print((en_time - st_time).total_seconds())

151.447558


In [45]:
perc_good = np.squeeze(outa[:,:,0])
mean_lw = np.squeeze(outa[:,:,1])
std_lw = np.squeeze(outa[:,:,2])
mean_nd = np.squeeze(outa[:,:,3])
std_nd = np.squeeze(outa[:,:,4])
mean_vi = np.squeeze(outa[:,:,5])
std_vi = np.squeeze(outa[:,:,6])
mean_btd = np.squeeze(outa[:,:,7])
std_btd = np.squeeze(outa[:,:,8])
mean_mir = np.squeeze(outa[:,:,9])
std_mir = np.squeeze(outa[:,:,10])
mean_vid = np.squeeze(outa[:,:,11])
std_vid = np.squeeze(outa[:,:,12])

vis_window_arr = vi_rad_arr - (mean_vi + 1.5 * std_vi)
ndfi_window_arr = ndfi_arr - (mean_nd + 1.5 * std_nd)
lw_window_arr = -1 * (lwi_rad_arr - (mean_lw + 1.5 * std_lw))
btd_window_arr = btd_arr - (mean_btd + 1.5 * std_btd)
mir_window_arr = mir_bt_arr - (mean_mir + 1.5 * std_mir)
visdif_window_arr = vi_diff_arr - (mean_vid + 1.5 * std_vid)

In [46]:
save_output(scn, vis_window_arr, 'vis_window_arr', odir, 'VI1_DIFF')
save_output(scn, ndfi_window_arr, 'ndfi_window_arr', odir, 'VI1_DIFF')
save_output(scn, lw_window_arr, 'lw_window_arr', odir, 'VI1_DIFF')
save_output(scn, btd_window_arr, 'btd_window_arr', odir, 'VI1_DIFF')
save_output(scn, mir_window_arr, 'mir_window_arr', odir, 'VI1_DIFF')
save_output(scn, visdif_window_arr, 'visdif_window_arr', odir, 'VI1_DIFF')

CPLE_AppDefinedError: Deleting D:/sat_data/SEV_FIRE/vis_window_arr_20230824_054000.tif failed: Permission denied

In [48]:
outarr = da.where(stg1_output > 0, 1, 0)
outarr = da.where(ndfi_window_arr > 0, outarr, 0)
outarr = da.where(visdif_window_arr > 0, outarr, 0)
outarr = da.where(vis_window_arr < 0.1, outarr, 0)
outarr = da.where(lwi_bt_arr > 260, outarr, 0)
outarr = da.where(mir_window_arr > 0, outarr, 0)
outarr = da.where(lsm == 2, outarr, 0)

save_output(scn, outarr, 'outarr', odir, 'VI1_DIFF')
